# Your First Image Classifier: Using MLP to Classify Images
# Pre-processing

The purpose of this dataset is to correctly classify an image as containing a dog, cat, or panda.
Containing only 3,000 images, the Animals dataset is meant to be another **introductory** dataset
that we can quickly train a MLP model and obtain a comparative results.

Let's take the following steps:

1. Fetch Data (reuse of the previous project)
2. Pre-processing
3. Clean data

<center><img width="900" src="https://drive.google.com/uc?export=view&id=1haMB_Zt6Et9q9sPHxfuR4g3FT5QRXlTI"></center>


## Step 01: Setup

Start out by installing the experiment tracking library and setting up your free W&B account:


*   **pip install wandb** – Install the W&B library
*   **import wandb** – Import the wandb library
*   **wandb login** – Login to your W&B account so you can log all your metrics in one place

In [1]:
!pip install wandb -qU

     |████████████████████████████████| 1.9 MB 5.5 MB/s 
     |████████████████████████████████| 182 kB 32.1 MB/s 
     |████████████████████████████████| 162 kB 56.2 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 162 kB 31.1 MB/s 
     |████████████████████████████████| 158 kB 38.9 MB/s 
     |████████████████████████████████| 157 kB 37.7 MB/s 
     |████████████████████████████████| 157 kB 38.9 MB/s 
     |████████████████████████████████| 157 kB 37.4 MB/s 
     |████████████████████████████████| 157 kB 40.5 MB/s 
     |████████████████████████████████| 157 kB 33.2 MB/s 
     |████████████████████████████████| 157 kB 22.8 MB/s 
     |████████████████████████████████| 157 kB 37.4 MB/s 
     |████████████████████████████████| 156 kB 42.5 MB/s 


### Import Packages

In [2]:
# import the necessary packages
from imutils import paths
import logging
import os
import cv2
import numpy as np
import joblib
import tensorflow as tf
import wandb

In [3]:
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [18]:
# configure logging
# reference for a logging obj
logger = logging.getLogger()

# set level of logging
logger.setLevel(logging.INFO)

# create handlers
c_handler = logging.StreamHandler()
c_format = logging.Formatter(fmt="%(asctime)s %(message)s",datefmt='%d-%m-%Y %H:%M:%S')
c_handler.setFormatter(c_format)

# add handler to the logger
logger.handlers[0] = c_handler

## Step 02: Fetch Data

In [19]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"dataset": "animals",
  "project_name": "first_image_classifier",
  "artifact_name": "animals_raw_data:latest",
  "eda_name": "eda_animals"
}

In [6]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="thaisaraujom",project=args["project_name"], job_type="preprocessing")

# download the raw data from W&B
raw_data = run.use_artifact(args["artifact_name"])
data_dir = raw_data.download()
logger.info("Path: {}".format(data_dir))

wandb: Currently logged in as: thaisaraujom. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact animals_raw_data:latest, 187.97MB. 3000 files... 
wandb:   3000 of 3000 files downloaded.  
Done. 0:0:32.7
17-10-2022 00:44:13 Path: ./artifacts/animals_raw_data:v0


In [7]:
run.finish()

## Step 03 - Clean Data

### Project Config.

In [8]:
data_dir

'./artifacts/animals_raw_data:v0'

In [9]:
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"features": "clean_features",
  "target": "labels",
  "project_name": "mlp_classifier"
}

In [10]:
# open the W&B project created in the Fetch step
run = wandb.init(entity="thaisaraujom",project=args["project_name"], job_type="preprocessing")

### Loader and Preprocessing Classes

Source code based on **Rosebrock, Adrian. Deep Learning For Computer vision with Python, 2019** [link](https://pyimagesearch.com/deep-learning-computer-vision-python-book/)

In [11]:
# a basic simple preprocessor
class SimplePreprocessor:
	def __init__(self, width, height, inter=cv2.INTER_AREA):
		# store the target image width, height, and interpolation
		# method used when resizing
		self.width = width
		self.height = height
		self.inter = inter

	def preprocess(self, image):
		# resize the image to a fixed size, ignoring the aspect
		# ratio
		return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [12]:
# Building an image loader
class SimpleDatasetLoader:
  def __init__(self, preprocessors=None, logger=None):
		# store the image preprocessor
    self.preprocessors = preprocessors
    self.logger = logger

		# if the preprocessors are None, initialize them as an
		# empty list
    if self.preprocessors is None:
      self.preprocessors = []

  def load(self, imagePaths, verbose=-1):
		# initialize the list of features and labels
    data = []
    labels = []

		# loop over the input images
    for (i, imagePath) in enumerate(imagePaths):
			# load the image and extract the class label assuming
			# that our path has the following format:
			# /path/to/dataset/{class}/{image}.jpg
			# e.g "img example: ./artifacts/animals_raw_data:v0/dogs/dogs_00892.jpg"
			# imagePath.split(os.path.sep)[-2] will return "dogs"
      image = cv2.imread(imagePath)
      label = imagePath.split(os.path.sep)[-2]

      # check to see if our preprocessors are not None
      if self.preprocessors is not None:
				# loop over the preprocessors and apply each to
				# the image
        for p in self.preprocessors:
          image = p.preprocess(image)

			# treat our processed image as a "feature vector"
			# by updating the data list followed by the labels
      data.append(image)
      labels.append(label)
   
			# show an update every `verbose` images
      if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
        logger.info("[INFO] processed {}/{}".format(i + 1,len(imagePaths)))

		# return a tuple of the data and labels
    return (np.array(data), np.array(labels))

### Cleaning

In [13]:
# grab the list of images that we'll be describing
logger.info("[INFO] preprocessing images...")
imagePaths = list(paths.list_images(data_dir))

# initialize the image preprocessor, load the dataset from disk,
# and reshape the data matrix
sp = SimplePreprocessor(32, 32)
sdl = SimpleDatasetLoader(preprocessors=[sp],logger=logger)
(data, labels) = sdl.load(imagePaths, verbose=500)
# 32 x 32 x 3 = 3072
data = data.reshape((data.shape[0], 3072))

# show some information on memory consumption of the images
logger.info("[INFO] features matrix: {:.1f}MB".format(data.nbytes / (1024 * 1024)))
logger.info("Data shape: {}".format(data.shape))
logger.info("Label shape: {}".format(labels.shape))

17-10-2022 00:44:19 [INFO] preprocessing images...
17-10-2022 00:44:20 [INFO] processed 500/3000
17-10-2022 00:44:22 [INFO] processed 1000/3000
17-10-2022 00:44:23 [INFO] processed 1500/3000
17-10-2022 00:44:25 [INFO] processed 2000/3000
17-10-2022 00:44:28 [INFO] processed 2500/3000
17-10-2022 00:44:31 [INFO] processed 3000/3000
17-10-2022 00:44:31 [INFO] features matrix: 8.8MB
17-10-2022 00:44:31 Data shape: (3000, 3072)
17-10-2022 00:44:31 Label shape: (3000,)


### Dump the artifacts to disk and upload to W&B

In [14]:
# Save the feature artifacts using joblib
joblib.dump(data, args["features"])

# Save the target using joblib
joblib.dump(labels, args["target"])

logger.info("Dumping the clean data artifacts to disk")

17-10-2022 00:44:31 Dumping the clean data artifacts to disk


In [15]:
# clean data artifact
artifact = wandb.Artifact(args["features"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean features data"
                          )

logger.info("Logging clean data artifact")
artifact.add_file(args["features"])
run.log_artifact(artifact)

17-10-2022 00:44:31 Logging clean data artifact


In [16]:
# clean label artifact
artifact = wandb.Artifact(args["target"],
                          type="CLEAN_DATA",
                          description="A json file representing the clean target"
                          )

logger.info("Logging clean target artifact")
artifact.add_file(args["target"])
run.log_artifact(artifact)

17-10-2022 00:44:31 Logging clean target artifact


In [17]:
run.finish()